1. pair representations $h_v^{(0)}$  赋值子图中每个节点和源节点信息
2. Message $M_v^{(1)}$: {$h_v^{(0)}$, 所有指向v的节点x的$h_x$和(x, v)总结后的信息}
3. 总结所有Message

In [43]:
import pandas as pd

In [44]:
sample = pd.read_csv('sample.csv')
cite = pd.read_csv('sample_cite.csv')

In [45]:
# 第一步
src = sample[sample.pid == 10169]
tar = sample[sample.pid == 63486]
v_df = sample[sample.pid != src.pid.values[0]].reset_index(drop=True)

h_v = pd.DataFrame({
    'src_pid': [src.pid.values[0]] * len(v_df),
    'src_title': [src.title.values[0]] * len(v_df),
    'src_abs': [src['abs'].values[0]] * len(v_df),
    'src_label': [src.label.values[0]] * len(v_df),
    'v_pid': v_df.pid,
    'v_title': v_df.title,
    'v_abs': v_df['abs'],
    'v_label': v_df.label
})
h_v

,src_pid,src_title,src_abs,src_label,v_pid,v_title,v_abs,v_label
0,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,63486,Title: Evaluation and Selection of Biases in M...,"Abstract: In this introduction, we define the ...",Theory
1,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,2440,"Title: Quinlan, 1990 J.R. Quinlan. Learning lo...",Abstract: We describe a ranked-model semantics...,Rule_Learning
2,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,3231,Title: Irrelevant Features and the Subset Sele...,Abstract: We address the problem of finding a ...,Theory
3,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,954315,Title: Inductive Database Design,Abstract: When designing a (deductive) databas...,Rule_Learning
4,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,144330,Title: Utilizing Prior Concepts for Learning,Abstract: The inductive learning problem consi...,Theory
5,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,56115,Title: Improving Tactical Plans with Genetic A...,Abstract:,Genetic_Algorithms
6,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,72908,Title: Applications of a logical discovery engine,Abstract: The clausal discovery engine claudie...,Rule_Learning
7,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,83461,Title: Dlab: A Declarative Language Bias Forma...,Abstract: We describe the principles and funct...,Rule_Learning
8,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,17811,Title: #1 Robust Feature Selection Algorithms,Abstract: Selecting a set of features which is...,Genetic_Algorithms
9,10169,Title: Learning Boolean Concepts in the Presen...,"Abstract: In this paper, we address the proble...",Theory,37483,Title: Learning Approximate Control Rules Of H...,Abstract: One of the difficult problems in the...,Case_Based


In [46]:
src.pid, tar['pid'].values[0]

(3    10169
 Name: pid, dtype: int64,
 63486)

In [47]:
# Indicator

hv_list_orig = []
for i in range(len(h_v)):

    src_title, src_abs, src_label = h_v.loc[i, 'src_title'], h_v.loc[i, 'src_abs'], h_v.loc[i, 'src_label']
    tar_pid, tar_title, tar_abs, tar_label = h_v.loc[i, 'v_pid'], h_v.loc[i, 'v_title'], h_v.loc[i, 'v_abs'], h_v.loc[i, 'v_label']

    # if tar['pid'].values[0] == tar_pid:
    #     continue
    
    # print(tar_pid)

    Indicator_prompt = f"""
Given two papers information shown as follow:
paper1:
title:{src_title}
abstract:{src_abs}
label:{src_label}
paper2:
title:{tar_title}
abstract:{tar_abs}
label:{tar_label}
please summarize information of these two papers in no more than three sentences, and then generate some keywords of this information.
Do not generate ordinal words like first, second, third, just provide the title of paper when mention a paper.
Do not generate paper number like paper1, paper2, just provide the title of paper when mention a paper.
"""

    hv_list_orig.append(Indicator_prompt)

In [48]:
for hv in hv_list_orig:
    print(hv)
    print('======================================================================================')


Given two papers information shown as follow:
paper1:
title:Title: Learning Boolean Concepts in the Presence of Many Irrelevant Features
abstract:Abstract: In this paper, we address the problem of case-based learning in the presence of irrelevant features. We review previous work on attribute selection and present a new algorithm, Oblivion, that carries out greedy pruning of oblivious decision trees, which effectively store a set of abstract cases in memory. We hypothesize that this approach will efficiently identify relevant features even when they interact, as in parity concepts. We report experimental results on artificial domains that support this hypothesis, and experiments with natural domains that show improvement in some cases but not others. In closing, we discuss the implications of our experiments, consider additional work on irrelevant features, and outline some directions for future research.
label:Theory
paper2:
title:Title: Evaluation and Selection of Biases in Machine 

In [49]:
# w(x, r, v)
# cite, src, tar
w_list_orig = []
for i in range(len(cite)):
    src_id, tar_id = cite.loc[i, 'src_id'], cite.loc[i, 'tar_id']

    if src.pid.values[0] == src_id and tar.pid.values[0] == tar_id:
        # print(src_id, tar_id)
        continue
    
    src_title, src_abs, src_label = sample[sample.pid == src_id].title.values[0], sample[sample.pid == src_id]['abs'].values[0], sample[sample.pid == src_id].label.values[0]
    tar_title, tar_abs, tar_label = sample[sample.pid == tar_id].title.values[0], sample[sample.pid == tar_id]['abs'].values[0], sample[sample.pid == tar_id].label.values[0]
    

    w_prompt = f"""
{src_id} {tar_id}
given two papers and their citation relationship information:
paper1:
title:{src_title}
abstract:{src_abs}
label:{src_label}
cites
paper2:
title:{tar_title}
abstract:{tar_abs}
label:{tar_label}
please summarize the citation relationship in no more than three sentences and then generate some keywords of this information.
    
"""

    w_list_orig.append(w_prompt)

for w in w_list_orig:
    print(w)
    print('=========================================================================')


2440 72908
given two papers and their citation relationship information:
paper1:
title:Title: Quinlan, 1990 J.R. Quinlan. Learning logical definitions from relations. Machine Learning, First-order theory revision. In
abstract:Abstract: We describe a ranked-model semantics for if-then rules admitting exceptions, which provides a coherent framework for many facets of evidential and causal reasoning. Rule priorities are automatically extracted form the knowledge base to facilitate the construction and retraction of plausible beliefs. To represent causation, the formalism incorporates the principle of Markov shielding which imposes a stratified set of independence constraints on rankings of interpretations. We show how this formalism resolves some classical problems associated with specificity, prediction and abduction, and how it offers a natural way of unifying belief revision, belief update, and reasoning about actions.
label:Rule_Learning
cites
paper2:
title:Title: Applications of a l

In [51]:
# message

src_id = src.pid.values[0]
tar_id = tar.pid.values[0]

message = {}
for i in range(len(sample)):

    node_id = sample.loc[i, 'pid']

    if node_id == src_id:
        continue


    h0 = sample.loc[i, 'h0']
    message[node_id] = {'h': [h0.replace('\n', '')]}

message

w
for key in message.keys():
    x_df = cite[cite.tar_id == key].reset_index(drop = True)
    message[key]['w'] = []
    for i in range(len(x_df)):
        if x_df.loc[i, 'src_id'] == src_id and x_df.loc[i, 'tar_id'] == tar_id:
            continue

        message[key]['w'].append(x_df.loc[i, 'w'].replace('\r', '').replace('\n', ''))


message
    

{63486: {'h': ['"Learning Boolean Concepts in the Presence of Many Irrelevant Features" introduces the Oblivion algorithm for feature selection in case-based learning amidst irrelevant features, focusing on parity concepts. It presents experimental results on artificial and natural domains, discussing implications and future research directions. "Evaluation and Selection of Biases in Machine Learning" defines bias in ML systems, advocates for automated bias evaluation, selection methods, and summarizes recent research in the field.Keywords:Case-based LearningIrrelevant FeaturesOblivion AlgorithmParity ConceptsFeature SelectionExperimental ResultsBias in Machine LearningAutomated Bias EvaluationBias SelectionMeta-bias SpacesResearch Summary'],
  'w': ['paper1:title:Title: Irrelevant Features and the Subset Selection Problemabstract:Abstract: We address the problem of finding a subset of features that allows a supervised induction algorithm to induce small high-accuracy concepts. We exam